## Importamos las librerias con las que vamos a trabajar

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# 1) Cargar el dataset desde CSV
df = pd.read_csv("crop_recommendation_outlierless.csv")

# 2) Separar características (X) y etiqueta (y)
#    Asumiendo que las columnas son: N, P, K, temperature, humidity, ph, rainfall, label
X = df[["N", "P", "K", "temperature", "humidity", "ph", "rainfall"]]
y = df["label"]

# 3) Convertir las etiquetas (strings de cultivos) a números
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [3]:
# 4) División 70 % entrenamiento / 30 % prueba (estratificada para mantener proporción de clases)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.30,
    random_state=42,
    stratify=y_encoded
)

In [4]:
# 5) Definir dos clasificadores SVM multiclasé:
#    - Uno con kernel LINEAL
#    - Otro con kernel RBF
svm_linear = SVC(kernel="linear", random_state=42)
svm_rbf    = SVC(kernel="rbf", random_state=42)

In [5]:
# 6) Validación cruzada (cv=5) sobre el conjunto de entrenamiento
#    Usamos cross_val_score para obtener las puntuaciones en 5 folds
cv_scores_linear = cross_val_score(svm_linear, X_train, y_train, cv=5)
cv_scores_rbf    = cross_val_score(svm_rbf,    X_train, y_train, cv=5)

print("=== Validación Cruzada (5‐fold) en datos de ENTRENAMIENTO ===\n")

print("SVM LINEAL:")
print(" Scores por fold:       ", cv_scores_linear)
print(" Media de accuracy:     {:.4f}".format(cv_scores_linear.mean()))
print(" Desviación estándar:   {:.4f}".format(cv_scores_linear.std()))
print()

print("SVM RBF:")
print(" Scores por fold:       ", cv_scores_rbf)
print(" Media de accuracy:     {:.4f}".format(cv_scores_rbf.mean()))
print(" Desviación estándar:   {:.4f}".format(cv_scores_rbf.std()))
print("\n" + "="*60 + "\n")

=== Validación Cruzada (5‐fold) en datos de ENTRENAMIENTO ===

SVM LINEAL:
 Scores por fold:        [0.98455598 0.97297297 0.98449612 0.98449612 0.98449612]
 Media de accuracy:     0.9822
 Desviación estándar:   0.0046

SVM RBF:
 Scores por fold:        [0.96138996 0.97297297 0.98062016 0.9496124  0.96511628]
 Media de accuracy:     0.9659
 Desviación estándar:   0.0105




In [6]:
# 7) Entrenar cada modelo con TODO el set de entrenamiento
svm_linear.fit(X_train, y_train)
svm_rbf.fit(X_train,    y_train)

SVC(random_state=42)

In [7]:
# 8) Hacer predicción sobre el conjunto de prueba (30 %)
y_pred_linear = svm_linear.predict(X_test)
y_pred_rbf    = svm_rbf.predict(X_test)

In [8]:
# 9) Calcular accuracy en prueba
accuracy_linear = accuracy_score(y_test, y_pred_linear)
accuracy_rbf    = accuracy_score(y_test, y_pred_rbf)

In [9]:
print("=== Resultados en CONJUNTO de PRUEBA (30 %) ===\n")
print(f"SVM LINEAL: Test Accuracy = {accuracy_linear:.4f}")
print(f"SVM RBF:    Test Accuracy = {accuracy_rbf:.4f}")
print("\n" + "="*60 + "\n")

# 10) Mostrar reporte de clasificación (precision / recall / f1-score) para cada kernel
print("Reporte de clasificación para SVM LINEAL:\n")
print(classification_report(
    y_test, 
    y_pred_linear, 
    target_names=label_encoder.classes_,
    zero_division=0
))

print("Reporte de clasificación para SVM RBF:\n")
print(classification_report(
    y_test, 
    y_pred_rbf, 
    target_names=label_encoder.classes_,
    zero_division=0
))

=== Resultados en CONJUNTO de PRUEBA (30 %) ===

SVM LINEAL: Test Accuracy = 0.9856
SVM RBF:    Test Accuracy = 0.9675


Reporte de clasificación para SVM LINEAL:

              precision    recall  f1-score   support

      banana       1.00      1.00      1.00        30
   blackgram       0.97      1.00      0.98        30
    chickpea       1.00      1.00      1.00        26
     coconut       1.00      1.00      1.00        30
      coffee       1.00      1.00      1.00        30
      cotton       1.00      1.00      1.00        30
        jute       0.87      0.90      0.89        30
 kidneybeans       1.00      1.00      1.00        30
      lentil       1.00      1.00      1.00        30
       maize       1.00      1.00      1.00        30
       mango       1.00      1.00      1.00        29
   mothbeans       1.00      0.94      0.97        18
    mungbean       1.00      1.00      1.00        30
   muskmelon       1.00      1.00      1.00        30
      orange       1.00  